In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Dec 14 03:54:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0    31W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [3]:
import glob

def preprocessing(input_path):
    all_input_list = sorted(glob.glob(input_path))
    train = pd.DataFrame()
    for datapath in all_input_list:
        data = pd.read_csv(datapath) 
  
        data['obs_time'] = data.index % 24 
        df = abs(data)
        df['시간당총광량'] = df['시간당청색광량']+df['시간당백색광량']+df['시간당적색광량']
        
        col_list = df.columns
        for i in range(0,len(col_list)):
            col = col_list[i]    
            if '누적' in col : 
                df[col] = df.groupby((df.obs_time == 0).cumsum()).agg(col_list[i-1]).cumsum()   
            df.to_csv(datapath,index=False)
            train = pd.concat([train,df])
    print('finish!!')
    return train

In [6]:
trainx = preprocessing('drive/MyDrive/dacon/gan/상추/train_input/*.csv')

finish!!


In [7]:
day0 = trainx[trainx['DAT']==0].reset_index(drop=True)
day0 = day0[['obs_time','내부온도관측치','내부습도관측치','co2관측치','ec관측치','시간당분무량','시간당백색광량','시간당적색광량','시간당청색광량']]

In [ ]:
pip install sdv

In [21]:
## sdv ##
from sdv.tabular import GaussianCopula # Importing 'GaussianCoupla' Model
from sdv.tabular import CTGAN # Importing 'CTGAN' Model
from sdv.tabular import CopulaGAN # Importing 'CopulaGAN' Model
from sdv.evaluation import evaluate # Importing 'evaluate' Class for Synthetic Dataset Evaluation

## seed ##
import numpy as np
import torch
import torch.backends.cudnn as cudnn
import random
np.random.seed(1)
torch.manual_seed(1)
random.seed(1)
torch.cuda.manual_seed(1)
torch.cuda.manual_seed_all(1)

**Synthetic Data Generation**

1. Gaussian Copula Model

In [10]:
syn_model = GaussianCopula() # Initializing the Model
syn_model.fit(day0) # Applying the Model on the Data

In [11]:
syn_model.save('drive/MyDrive/dacon/gan/Gaussian.pkl')

In [12]:
gaussian = GaussianCopula.load('drive/MyDrive/dacon/gan/Gaussian.pkl')

In [15]:
day0.describe()

,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,시간당백색광량,시간당적색광량,시간당청색광량
count,10752.000000,10752.000000,10752.000000,10752.000000,10752.000000,10752.000000,10752.000000,10752.000000,10752.000000
mean,11.500000,25.781004,54.912360,533.833783,1.273970,430.609479,6765.408458,1309.564887,856.852189
std,6.922508,4.361793,12.257845,144.127605,0.932456,491.308581,9450.283141,2653.722924,1938.172567
min,0.000000,0.000000,0.000000,60.400000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.750000,23.595463,49.183641,446.345833,0.401136,0.000000,0.000000,0.000000,0.000000
50%,11.500000,26.280460,56.510000,505.883333,1.212455,242.355000,0.000000,0.000000,0.000000
75%,17.250000,28.717917,61.310767,578.400000,2.011237,769.000000,18255.190000,976.332000,135.110625
max,23.000000,39.158823,81.900001,1200.000000,3.034100,2735.210000,120000.000000,9928.800000,18570.857500


In [24]:
syn_data = gaussian.sample(num_rows = 1000) # Generating Synthetic Instances
syn_data.describe()

,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,시간당백색광량,시간당적색광량,시간당청색광량
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,11.590000,26.093834,55.105524,533.572000,1.310000,529.951000,9595.769000,2314.250000,1647.242000
std,6.671012,4.503089,12.283833,142.688769,0.929927,402.316001,7411.925424,1733.755625,1238.593713
min,0.000000,10.798987,18.394126,150.000000,0.000000,0.000000,2.000000,17.000000,0.000000
25%,6.000000,23.030654,46.179191,436.000000,1.000000,212.000000,3624.500000,870.000000,644.750000
50%,12.000000,26.334400,55.301737,535.000000,1.000000,442.000000,8151.500000,2050.500000,1363.000000
75%,17.000000,29.364403,64.053269,632.000000,2.000000,760.500000,14095.250000,3359.000000,2450.500000
max,23.000000,37.974050,81.816704,1024.000000,3.000000,2044.000000,46359.000000,8008.000000,7487.000000


1-1 ) Evaluating Synthetic Instances : Statistical Metrics

In [30]:
?evaluate

In [27]:
general_evaluation = evaluate(syn_data, day0) # Evaluating on All Metrics
CS_test = evaluate(syn_data, day0, metrics = ['CSTest']) # Chi-Squared Test
#KS_test = evaluate(syn_data, day0, metrics = ['KSTest']) # Inverted Kolmogorov-Smirnov Test

In [28]:
evaluation = [general_evaluation, CS_test] # Creating a List of the Results
evaluation = pd.DataFrame(evaluation) # Creating a Data Frame of the Results
evaluation.columns = ['Results'] # Gicing Column Name
evaluation.rename({0:'General Evaluation', 1:'Chi-Squared Test', 2:'Inverted Kolmogorov-Smirnov Test'}) # Renaming Row Name

,Results
General Evaluation,0.672397
Chi-Squared Test,NaN
